# Задачка с собеседований

In [1]:
import pandas as pd
import sqlite3

In [2]:
con = sqlite3.connect('finance.db')

In [3]:
pd.read_sql("""
    select date, account, value from debit
    """, con)

,date,account,value
0,2021-01-01,111,100
1,2021-01-01,222,200
2,2021-01-01,111,110
3,2021-01-02,111,90
4,2021-01-03,111,90


In [4]:
pd.read_sql("""
    select date, account, value from credit
    """, con)

,date,account,value
0,2021-01-01,111,50
1,2021-01-01,222,100
2,2021-01-01,111,55
3,2021-01-02,111,45


### Ошибочное объединение

In [5]:
pd.read_sql("""
    select d.date, d.account, d.value, c.value from debit d
    left outer join credit c
    on d.date=c.date and d.account = c.account
    """, con)

,date,account,value,value
0,2021-01-01,111,100,50.0
1,2021-01-01,111,100,55.0
2,2021-01-01,222,200,100.0
3,2021-01-01,111,110,50.0
4,2021-01-01,111,110,55.0
5,2021-01-02,111,90,45.0
6,2021-01-03,111,90,NaN


Добавляем группировку

In [6]:
pd.read_sql("""
    select d.date, d.account, d.value from (
        select date, account, sum(value) as value
        from debit
        group by date, account
    ) d
    """, con)

,date,account,value
0,2021-01-01,111,210
1,2021-01-01,222,200
2,2021-01-02,111,90
3,2021-01-03,111,90


In [7]:
pd.read_sql("""
    select c.date, c.account, c.value from (
        select date, account, sum(value) as value
        from credit
        group by date, account
    ) c
    """, con)

,date,account,value
0,2021-01-01,111,105
1,2021-01-01,222,100
2,2021-01-02,111,45


И только после группировки объединяем

In [8]:
pd.read_sql("""
    select d.date, d.account, d.value from (
        select date, account, sum(value) as value
        from debit
        group by date, account
    ) d
    
    left outer join
    
    (select c.date, c.account, c.value from (
        select date, account, sum(value) as value
        from credit
        group by date, account
    ) c) c_2
    
    on d.date=c_2.date and d.account=c_2.account
    """, con)

,date,account,value
0,2021-01-01,111,210
1,2021-01-01,222,200
2,2021-01-02,111,90
3,2021-01-03,111,90


### Вариант получше через UNION

In [9]:
pd.read_sql("""
    select date, account, sum(value) 
    from debit
    group by date, account
    
    union all
    
    select date, account, -sum(value) 
    from credit
    group by date, account
    """, con)

,date,account,sum(value)
0,2021-01-01,111,210
1,2021-01-01,222,200
2,2021-01-02,111,90
3,2021-01-03,111,90
4,2021-01-01,111,-105
5,2021-01-01,222,-100
6,2021-01-02,111,-45


In [10]:
pd.read_sql(""" 
    select date, account, sum(value) from (
        select date, account, sum(value) as value
        from debit
        group by date, account

        union all

        select date, account, -sum(value) as value
        from credit
        group by date, account
    )
    group by date, account
    """, con)

,date,account,sum(value)
0,2021-01-01,111,105
1,2021-01-01,222,100
2,2021-01-02,111,45
3,2021-01-03,111,90


In [11]:
con.close()

# Простая база данных

Запись датафрейма в базу данных

In [12]:
import pandas as pd
import sqlite3

In [ ]:
df = pd.read_csv('keywords.csv')
df.head()

In [ ]:
con = sqlite3.connect('keywords.db')

In [ ]:
df.to_sql('keywords', con, if_exists='replace')

In [ ]:
con.close()

Чтение из базы

In [ ]:
con = sqlite3.connect('keywords.db')

In [ ]:
df_sql = pd.read_sql('select * from keywords where keyword like "%вконтакте%";', con)
df_sql

Построчная обработка

In [ ]:
cur = con.cursor()

In [ ]:
cur.execute('select * from keywords where keyword like "%вконтакте%";')

In [ ]:
line = cur.fetchone()
line

In [ ]:
another_line = cur.fetchone()
another_line

In [ ]:
for i, line in enumerate(cur.execute('select * from keywords where keyword like "%вконтакте%";')):
    print(line)
    
    if i > 5:
        break